In [ ]:
# ! pip install datasets transformers sacrebleu accelerate

In [6]:
import pandas as pd
import numpy as np
import datasets
import warnings
warnings.filterwarnings("ignore")

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric, Dataset

In [8]:
# load the datasets
train = pd.read_csv("/content/drive/MyDrive/text-detoxification/data/interim/train.csv")
test = pd.read_csv("/content/drive/MyDrive/text-detoxification/data/interim/test.csv")

In [9]:
datasets = datasets.DatasetDict({"train":Dataset.from_pandas(train),"test":Dataset.from_pandas(test)})

In [10]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

#### get tokens from reference and translated sentences

In [11]:
# def preprocess_function(dataset):
#     model_inputs = tokenizer(dataset["reference"], truncation=True, max_length=128)
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(dataset["translation"], truncation=True, max_length=128)

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

prefix = 'detoxify: '
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')

batch_size = 16
args = Seq2SeqTrainingArguments(
    f"/content/drive/MyDrive/text-detoxification/models/t5-base",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=4,
    eval_accumulation_steps=1,
    predict_with_generate=True,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=True,
    num_train_epochs=1,
    save_steps=10000,
    report_to="none"
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### define metrics calculators

In [15]:
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [16]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.394800,1.277130,27.564500,13.000600


TrainOutput(global_step=10000, training_loss=1.4443621765136718, metrics={'train_runtime': 7362.8796, 'train_samples_per_second': 21.731, 'train_steps_per_second': 1.358, 'total_flos': 8437416052654080.0, 'train_loss': 1.4443621765136718, 'epoch': 1.0})

In [18]:
model.save_pretrained("/content/drive/MyDrive/text-detoxification/models/t5-base/trained")